<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1hc_tb7sMdDOa7fTMAe2Fu1wslv1wf9E8#scrollTo=ahguSJ5vp3e7)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
* Empowering the Next Generation of AI Innovators

##Getting Started With LangGraoh

###**Introduction 🔖**

LangGraph is a library built on top of LangChain that helps create stateful, multi-step AI workflows. Think of it as a way to create flowcharts for AI tasks, where each step can depend on previous results and make decisions about what to do next.

### Basic Concepts
1. Nodes: Individual steps in your workflow
2. Edges: Connections between steps
3. State: Information that persists between steps

## Setup
To get started, we need to install the last version of langgraph

In [6]:
!pip install langchain langchain_openai langchain-community langgraph --quiet

### Key Features :-

- Create multi-step AI workflows
- Maintain state between steps
- Conditional branching
- Easy integration with LLMs

### Greeting Workflow

This workflow takes a person's name and generates a greeting message.

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

# Define our state
class GreetingState(TypedDict):
    name: str
    greeting: str

# Create nodes
def create_greeting(state):
    # Create a greeting message using the name
    state['greeting'] = f"Hello, {state['name']}!"
    return state

In [3]:
# Create workflow
greeting_workflow = StateGraph(GreetingState)

# Add nodes
greeting_workflow.add_node("greet", create_greeting)

# Add edges
greeting_workflow.add_edge(START, "greet")  # Start the workflow
greeting_workflow.add_edge("greet", END)    # End the workflow

# Compile
chain = greeting_workflow.compile()


###Run the Chain

In [4]:
# Run
result = chain.invoke({
    "name": "Alice",
    "greeting": ""
})
print(result)

{'name': 'Alice', 'greeting': 'Hello, Alice!'}


###Setting Up API Keys

In [7]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

###Create a Node

In [8]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
import os

# Define our state
class OpenAIState(TypedDict):
    prompt: str
    response: str

# Create nodes
def generate_response(state):
    # Use OpenAI to generate a response
    llm = ChatOpenAI(temperature=0)
    response = llm.invoke(state['prompt'])
    state['response'] = response.content
    return state

In [9]:
# Create workflow
openai_workflow = StateGraph(OpenAIState)

# Add nodes
openai_workflow.add_node("generate", generate_response)

# Add edges
openai_workflow.add_edge(START, "generate")  # Start the workflow
openai_workflow.add_edge("generate", END)    # End the workflow

# Compile
chain = openai_workflow.compile()

###Run the Chain

In [10]:
# Run
result = chain.invoke({
    "prompt": "What is the capital of France?",
    "response": ""
})
print(result)

{'prompt': 'What is the capital of France?', 'response': 'The capital of France is Paris.'}


In [11]:
result = chain.invoke({
    "prompt": "Highest mountain in India",
    "response": ""
})
print(result)

{'prompt': 'Highest mountain in India', 'response': 'The highest mountain in India is Kangchenjunga, which stands at an elevation of 8,586 meters (28,169 feet) above sea level. It is located on the border between Nepal and the Indian state of Sikkim in the eastern Himalayas.'}


## Common Use Cases 🎢
- Customer Support Automation
- Content Generation
- Research Assistance
- Data Analysis Pipelines
- Decision Support Systems

### A real life scenario and usecase

This code creates an interactive customer support system using LangGraph and OpenAI. It simulates how a support agent might handle a customer's issue by categorizing the problem, determining its priority, and providing a solution.



In [12]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
from langchain_openai import ChatOpenAI
import os
from IPython.display import clear_output
import time

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Define our state
class TicketState(TypedDict):
    customer_name: str
    issue: str
    category: str
    priority: str
    solution: str
    satisfaction: str
    chat_history: List[str]


### Helper node functions

In [13]:
# Node functions
def categorize_issue(state):
    llm = ChatOpenAI(temperature=0)
    prompt = f"""
    Categorize this customer issue into one of:
    - Technical Support
    - Billing Query
    - Product Information
    - General Inquiry

    Issue: {state['issue']}

    Respond with just the category name.
    """
    state['category'] = llm.invoke(prompt).content.strip()
    state['chat_history'].append(f"System: Categorized as {state['category']}")
    return state

def determine_priority(state):
    llm = ChatOpenAI(temperature=0)
    prompt = f"""
    Based on this customer issue, determine the priority (High/Medium/Low):
    Customer: {state['customer_name']}
    Issue: {state['issue']}
    Category: {state['category']}

    Consider:
    - Impact on customer
    - Urgency of the matter
    - Category of issue

    Respond with just: High, Medium, or Low
    """
    state['priority'] = llm.invoke(prompt).content.strip()
    state['chat_history'].append(f"System: Priority set to {state['priority']}")
    return state

def generate_solution(state):
    llm = ChatOpenAI(temperature=0.7)
    prompt = f"""
    Generate a helpful and professional solution for this customer:

    Customer: {state['customer_name']}
    Category: {state['category']}
    Priority: {state['priority']}
    Issue: {state['issue']}

    Provide a clear, step-by-step solution.
    """
    state['solution'] = llm.invoke(prompt).content.strip()
    state['chat_history'].append(f"Agent: {state['solution']}")
    return state

def get_satisfaction(state):
    while True:
        satisfaction = input("\nAre you satisfied with the solution? (yes/no): ").lower()
        if satisfaction in ['yes', 'no']:
            state['satisfaction'] = satisfaction
            state['chat_history'].append(f"Customer: Satisfaction - {satisfaction}")
            break
        print("Please enter 'yes' or 'no'")
    return state

def handle_dissatisfaction(state):
    if state['satisfaction'] == 'no':
        llm = ChatOpenAI(temperature=0.7)
        prompt = f"""
        The customer is not satisfied with the previous solution.
        Generate an alternative solution:

        Original Issue: {state['issue']}
        Previous Solution: {state['solution']}

        Provide a different approach to solve the issue.
        """
        new_solution = llm.invoke(prompt).content.strip()
        state['solution'] = new_solution
        state['chat_history'].append(f"Agent: I apologize for the previous solution. Here's an alternative approach:\n{new_solution}")
    return state

### Workflow creation
Creating a workflow for Customer Support Tickting System

In [14]:
# Create the workflow
def create_support_workflow():
    workflow = StateGraph(TicketState)

    # Add nodes
    workflow.add_node("categorize", categorize_issue)
    workflow.add_node("prioritize", determine_priority)
    workflow.add_node("solve", generate_solution)
    workflow.add_node("check_satisfaction", get_satisfaction)
    workflow.add_node("handle_dissatisfaction", handle_dissatisfaction)

    # Add edges
    workflow.add_edge(START, "categorize")
    workflow.add_edge("categorize", "prioritize")
    workflow.add_edge("prioritize", "solve")
    workflow.add_edge("solve", "check_satisfaction")
    workflow.add_edge("check_satisfaction", "handle_dissatisfaction")
    workflow.add_edge("handle_dissatisfaction", END)

    return workflow.compile()

def display_ticket_status(state):
    """Display the current status of the ticket"""
    clear_output(wait=True)
    print("\n=== Customer Support Ticket Status ===")
    print(f"Customer Name: {state['customer_name']}")
    print(f"Category: {state['category']}")
    print(f"Priority: {state['priority']}")
    print("\n=== Chat History ===")
    for message in state['chat_history']:
        print(message)
        time.sleep(1)  # Add a small delay for better readability

def run_support_system():
    """Main function to run the support system"""
    print("Welcome to the Customer Support System!")

    # Get customer information
    customer_name = input("\nPlease enter your name: ")
    issue = input("Please describe your issue: ")

    # Initialize the support workflow
    chain = create_support_workflow()

    # Initial state
    initial_state = {
        "customer_name": customer_name,
        "issue": issue,
        "category": "",
        "priority": "",
        "solution": "",
        "satisfaction": "",
        "chat_history": [f"Customer: {issue}"]
    }

    # Run the workflow
    print("\nProcessing your ticket...")
    result = chain.invoke(initial_state)

    # Display final status
    display_ticket_status(result)

    # Ask if they want to create another ticket
    if input("\nWould you like to create another ticket? (yes/no): ").lower() == 'yes':
        run_support_system()
    else:
        print("Thank you for using our support system!")


### Run the system

* In this Example We are Using Custumer Support Ticket System


In [15]:
# Run the system
if __name__ == "__main__":
    run_support_system()


=== Customer Support Ticket Status ===
Customer Name: shubham
Category: Technical Support
Priority: High

=== Chat History ===
Customer: getting error while contacting
System: Categorized as Technical Support
System: Priority set to High
Agent: Step 1: Check your internet connection to ensure it is stable and working properly.

Step 2: Make sure you have entered the correct contact information for the person you are trying to reach.

Step 3: Try contacting the person using a different method, such as email or a different messaging platform, to see if the issue is specific to one method of communication.

Step 4: If you are still experiencing issues, try restarting your device and/or clearing the cache of the app you are using to contact the person.

Step 5: If the issue persists, reach out to the customer support team of the app or platform you are using for further assistance. Provide them with details of the error message you are receiving for a quicker resolution.

Step 6: If all e